# <font color='blue'>Data Science Challenge @ ITA 2022</font>
# <font color='blue'>Equipe DIOMGIS</font>

## <font color='blue'>1º Fase</font>

### <font color='blue'>Predição de pregões futuros de ativos que compõem o índice SP500.</font>

![title](data\image\logo.jpeg)

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.12


In [2]:
# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.

!pip install -q -U watermark

In [19]:
# Bibliotecas e Frameworks

import numpy as np
import pandas as pd
import pandas_datareader.data as web
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential, load_model
from keras.optimizers import *
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from keras.losses import MeanSquaredError
from tensorboard import notebook
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
# from time import time
from datetime import datetime
import datetime as dt
import os

In [4]:
# Versões dos pacotes usados neste jupyter notebook

%reload_ext watermark
%watermark -a "Equipe DIOMGIS" --iversions

Author: Equipe DIOMGIS

numpy            : 1.22.3
pandas_datareader: 0.10.0
tensorboard      : 2.10.0
pandas           : 1.4.2
seaborn          : 0.11.2
matplotlib       : 3.5.1
keras            : 2.10.0
tensorflow       : 2.10.0



In [5]:
# Parametros fixos

verbose = 2
seed = 25
steps = 30
epochs = 1000
batch_size = 32
logRetPeriod = 20
graphic = False
downloadData = False
trainModel = False

In [6]:
startDate = '2017-10-21'
endDate = '2022-10-18'

In [7]:
forecast = ['2022-10-20', '2022-10-21', '2022-10-24', '2022-10-24', '2022-10-25', '2022-10-26', '2022-10-27', '2022-10-28', 
            '2022-10-31', '2022-11-01', '2022-11-02', '2022-11-03', '2022-11-04', 
            '2022-11-07', '2022-11-08', '2022-11-09', '2022-11-10', '2022-11-11',
            '2022-11-14', '2022-11-15', '2022-11-16', '2022-11-17', '2022-11-18']

In [8]:
sns.set_style('whitegrid')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
%matplotlib inline
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
plt.rcParams['figure.figsize'] = (15, 10)
np.random.seed(seed)

In [9]:
#Confirma se o TensorFlow pode acessar a GPU

device_name = tf.test.gpu_device_name()
if not device_name:
    raise SystemError('GPU device not found')
    
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [10]:
# Estado da GPU

!nvidia-smi

Wed Oct 19 23:59:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.94       Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:65:00.0  On |                  N/A |
|  0%   49C    P2    28W / 220W |   1247MiB /  8192MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
setores = {
    'Industrials': ['MMM', 'AOS', 'ALK', 'ALLE', 'AAL', 'AME', 'BA', 'CHRW', 'CARR', 'CAT', 'CTAS', 'CPRT', 'CSGP',
                    'CSX', 'CMI', 'DE', 'DAL', 'DOV', 'ETN', 'EMR', 'EFX', 'EXPD', 'FAST', 'FDX', 'FTV', 'FBHS',
                    'GNRC', 'GD', 'GE', 'HON', 'HWM', 'HII', 'IEX', 'ITW', 'IR', 'JBHT', 'J', 'JCI', 'LHX', 'LDOS',
                    'LMT', 'MAS', 'NDSN', 'NSC', 'NOC', 'ODFL', 'OTIS', 'PCAR', 'PH', 'PNR', 'PWR', 'RTX', 'RSG',
                    'RHI', 'ROK', 'ROL', 'SNA', 'LUV', 'SWK', 'TXT', 'TT', 'TDG', 'UNP', 'UAL', 'UPS', 'URI', 'VRSK',
                    'WAB', 'WM', 'GWW', 'XYL'],
    
    'HealthCare': ['ABT', 'ABBV', 'ABMD', 'A', 'ALGN', 'ABC', 'AMGN', 'BAX', 'BDX', 'BIO', 'TECH', 'BIIB', 'BSX', 'BMY',
                   'CAH', 'CTLT', 'CNC', 'CRL', 'CI', 'COO', 'CVS', 'DHR', 'DVA', 'XRAY', 'DXCM', 'EW', 'ELV', 'LLY',
                   'GILD', 'HCA', 'HSIC', 'HOLX', 'HUM', 'IDXX', 'ILMN', 'INCY', 'ISRG', 'IQV', 'JNJ', 'LH', 'MCK',
                   'MDT', 'MRK', 'MTD', 'MRNA', 'MOH', 'OGN', 'PKI', 'PFE', 'DGX', 'REGN', 'RMD', 'STE', 'SYK', 'TFX',
                   'TMO', 'UNH', 'UHS', 'VRTX', 'VTRS', 'WAT', 'WST', 'ZBH', 'ZTS'],
    
    'InformationTechnology': ['ACN', 'ADBE', 'ADP', 'AKAM', 'AMD', 'APH', 'ADI', 'ANSS', 'AAPL', 'AMAT', 'ANET', 'ADSK',
                              'AVGO', 'BR', 'CDNS', 'CDW', 'CDAY', 'CSCO', 'CTSH', 'GLW', 'DXC', 'ENPH', 'EPAM', 'FFIV',
                              'FIS', 'FISV', 'FLT', 'FTNT', 'IT', 'GPN', 'HPE', 'HPQ', 'IBM', 'INTC', 'INTU', 'JKHY',
                              'JNPR', 'KEYS', 'KLAC', 'LRCX', 'MA', 'MCHP', 'MU', 'MSFT', 'MPWR', 'MSI', 'NTAP', 'NLOK',
                              'NVDA', 'NXPI', 'ON', 'ORCL', 'PAYX', 'PAYC', 'PYPL', 'PTC', 'QRVO', 'QCOM', 'ROP', 'CRM',
                              'STX', 'NOW', 'SWKS', 'SEDG', 'SNPS', 'TEL', 'TDY', 'TER', 'TXN', 'TRMB', 'TYL', 'VRSN',
                              'V', 'WDC', 'ZBRA'],
    
    'CommunicationServices': ['ATVI', 'GOOGL', 'GOOG', 'T', 'CHTR', 'CMCSA', 'DISH', 'DIS', 'EA', 'FOXA', 'FOX', 'IPG',
                              'LYV', 'LUMN', 'MTCH', 'META', 'NFLX', 'NWSA', 'NWS','OMC', 'PARA', 'TMUS', 'TTWO', 'TWTR',
                              'VZ', 'WBD'],
    
    'ConsumerStaples': ['ADM', 'MO', 'BF.B', 'CPB', 'CHD', 'CLX', 'KO', 'CL', 'CAG', 'STZ', 'COST', 'EL', 'GIS', 'HSY',
                        'HRL', 'K', 'KDP', 'KMB', 'KHC', 'KR', 'LW', 'MKC', 'TAP', 'MDLZ', 'MNST', 'PEP', 'PM', 'PG',
                        'SJM', 'SYY', 'TSN', 'WBA', 'WMT'],
    
    'ConsumerDiscretionary': ['AAP', 'AMZN', 'APTV', 'AZO', 'BBWI', 'BBY', 'BKNG', 'BWA', 'CZR', 'KMX', 'CCL', 'CMG',
                              'DHI', 'DRI', 'DG', 'DLTR', 'DPZ', 'EBAY', 'ETSY', 'EXPE', 'F','GRMN', 'GM', 'GPC', 'HAS',
                              'HLT', 'HD', 'LVS', 'LEN', 'LKQ', 'LOW', 'MAR', 'MCD', 'MGM', 'MHK', 'NWL', 'NKE', 'NCLH',
                              'NVR', 'ORLY', 'POOL', 'PHM', 'RL', 'ROST', 'RCL', 'SBUX', 'TPR', 'TGT', 'TSLA', 'TJX',
                              'TSCO', 'ULTA', 'VFC', 'WHR', 'WYNN', 'YUM'],
    
    'Utilities': ['AES', 'LNT', 'AEE', 'AEP', 'AWK', 'ATO', 'CNP', 'CMS', 'ED', 'CEG', 'D', 'DTE', 'DUK', 'EIX','ETR',
                  'EVRG', 'ES', 'EXC', 'FE', 'NEE', 'NI', 'NRG', 'PCG', 'PNW', 'PPL', 'PEG', 'SRE', 'SO', 'WEC', 'XEL'],
    
    'Financials': ['AFL', 'ALL', 'AXP', 'AIG', 'AMP', 'AON', 'AJG', 'AIZ', 'BAC', 'WRB', 'BRK.B', 'BLK', 'BK', 'BRO',
                   'COF', 'CBOE', 'SCHW', 'CB', 'CINF', 'C', 'CFG', 'CME', 'CMA', 'DFS', 'RE', 'FDS', 'FITB', 'FRC',
                   'BEN', 'GL', 'GS', 'HIG', 'HBAN', 'ICE', 'IVZ', 'JPM', 'KEY', 'LNC', 'L', 'MTB', 'MKTX', 'MMC',
                   'MET', 'MCO', 'MS', 'MSCI', 'NDAQ', 'NTRS', 'PNC', 'PFG', 'PGR', 'PRU', 'RJF', 'RF', 'SPGI', 'SBNY',
                   'STT', 'SIVB', 'SYF', 'TROW', 'TRV', 'TFC', 'USB', 'WFC', 'WTW', 'ZION',  'NLSN'],
    
    'Materials': ['APD', 'ALB', 'AMCR', 'AVY', 'BALL', 'CE', 'CF', 'CTVA', 'DOW', 'DD', 'EMN', 'ECL', 'FMC', 'FCX',
                  'IP', 'IFF', 'LIN', 'LYB', 'MLM', 'MOS', 'NEM', 'NUE', 'PKG', 'PPG', 'SEE', 'SHW', 'VMC', 'WRK'],
    
    'RealEstate': ['ARE', 'AMT', 'AVB', 'BXP', 'CPT', 'CBRE', 'CCI', 'DLR', 'EQIX', 'EQR', 'ESS', 'EXR', 'FRT', 'PEAK',
                   'HST', 'INVH', 'IRM', 'KIM', 'MAA', 'PLD', 'PSA', 'O', 'REG', 'SBAC', 'SPG', 'UDR', 'VTR', 'VICI',
                   'VNO', 'WELL', 'WY'],
    
    'Energy': ['APA','BKR', 'CVX', 'COP', 'CTRA', 'DVN', 'FANG', 'EOG', 'EQT', 'XOM', 'HAL', 'HES', 'KMI', 'MRO',
               'MPC', 'OXY', 'OKE', 'PSX', 'PXD', 'SLB', 'VLO', 'WMB']
}

In [12]:
ativos = []
for setor, empresas in setores.items():
    ativos.extend(empresas)
    
ativos.sort()

## Download dos Dados

In [13]:
if downloadData:

    data = web.DataReader(name = '^GSPC', data_source = 'yahoo', start = startDate, end = endDate)
    SP500_index = pd.DataFrame(data['Close']).reset_index().rename(columns={'Close': 'SP500', 'Date': 'Dia'})
    
    aux = []

    for ativo in ativos:
  
        if ativo == 'BF.B':
            ativo = 'BF-B'

        if ativo == 'BRK.B':
            ativo = 'BRK-B'

        data = web.DataReader(name = ativo, data_source = 'yahoo', start = startDate, end = endDate)
        temp_close = pd.DataFrame(data['Close'])
        
        aux.append(temp_close)       

    SP500_close = pd.concat(aux, axis = 1).copy()
    SP500_close.columns = ativos
    SP500_close.reset_index(inplace = True)
    SP500_close.rename(columns={'Date': 'Dia'}, inplace = True)

    outdir = './data'

    if not os.path.exists(outdir):
        os.mkdir(outdir)
        
    SP500_close.to_csv(path_or_buf = os.path.join(outdir, 'SP500Close.csv'), index = False)
    SP500_index.to_csv(path_or_buf = os.path.join(outdir, 'SP500Index.csv'), index = False)
    SP500_close.to_excel(os.path.join(outdir, '.SP500Close.xlsx'))
    SP500_index.to_excel(os.path.join(outdir, '.SP500Index.xlsx'))
    
else:
    
    SP500_close = pd.read_csv('./data/SP500Close.csv')
    SP500_index = pd.read_csv('./data/SP500Index.csv')

## Pré-Processamento dos Dados

In [14]:
def generatorTimeframeTable(table, ativo):
    
    nameColumns = []

    for i in range(steps,-1,-1):
        nameColumns.append('Close-{}'.format(i))
    
    TimeframeTable = pd.DataFrame(np.zeros((len(table[ativo])-steps, steps+1), dtype='float64'), columns = nameColumns)

    for index, close in enumerate(table[ativo]):
        tempA = index
        tempB = 0
        for i in range(steps+1):
            if tempA < len(table[ativo])-steps and tempA >=0:
                TimeframeTable.iloc[tempA, tempB] = close

            tempA -= 1
            tempB += 1

    timeIndex = table.iloc[steps:,0]
    TimeframeTable["Dia"] = timeIndex.to_numpy()
    TimeframeTable.set_index("Dia", inplace = True)
    
    return TimeframeTable

In [15]:
def createTrainScaler(df):
    
    trainScaler = pd.DataFrame()
 
    for _ in range(steps+1):
        temp_close = pd.DataFrame(df.iloc[:,-1])
        trainScaler = pd.concat([trainScaler, temp_close], axis = 1)

    scaler = StandardScaler()

    scaler.fit(trainScaler)

    return scaler

In [16]:
def preprocessingdata(steps, df, ativos):
    
    nameColumns = []

    for i in range(steps,-1,-1):
        nameColumns.append('Close-{}'.format(i))
    

    aux = []
    
    for ativo in ativos:
        trainDataAtivo = generatorTimeframeTable(df, ativo)
        trainDataAtivo.dropna(axis = 0, inplace = True)
        
        #----Score-Z--------------------------------------
        scaler = createTrainScaler(trainDataAtivo)
        trainDataAtivo = scaler.transform(trainDataAtivo)
        #-------------------------------------------------
        aux.append(trainDataAtivo)
    
    trainData = np.concatenate(tuple(aux), axis=0)
    
    X = trainData[:, :-1]
    y = trainData[:, -1]
    

    #------Divisão de dados entre Treino e Validação------------------------------------------------
    
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.2, shuffle = False)

    X_treino = X_treino.reshape((-1, steps, 1))
    X_teste = X_teste.reshape((-1, steps, 1))
    #-----------------------------------------------------------------------------------------------
    
    return [X_treino, X_teste, y_treino, y_teste]

## Construção, Treinamento e Avaliação do Modelo

In [20]:
# Callbacks

tensorboard = TensorBoard(log_dir="logs/{}".format(datetime.now().strftime('%d-%B-%Ih%Mmin')))

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=20,
                          verbose = verbose,
                          restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='loss',
                              factor=0.2,
                              patience=3,
                              mode="min",
                              verbose = verbose,
                              min_delta=0.0001,
                              min_lr=0)

callbacks = [tensorboard, earlystop, reduce_lr, TerminateOnNaN()]

In [21]:
def create_model():
     
    model = Sequential()
    
    model.add(LSTM(160,
                   activation = 'tanh',
                   recurrent_activation = 'sigmoid',
                   return_sequences = True,
                   input_shape = (steps, 1)))  

    model.add(LSTM(160,
                   activation = 'tanh',
                   recurrent_activation = 'sigmoid',
                   return_sequences = True))  
    
    model.add(LSTM(160,
                   activation = 'tanh',
                   recurrent_activation = 'sigmoid',
                   return_sequences = False)) 
    
    model.add(Dense(1, activation = 'linear'))
    
    Lmse = MeanSquaredError()
    
    opt_Adadelta = Adadelta(learning_rate = 0.01, rho = 0.95, epsilon = 1e-07)

    model.compile(loss= Lmse, optimizer = opt_Adadelta)

    return model

In [22]:
def fillTableFrame(ativo, tablePrevision, model, table = SP500_close):

    TimeframeTable = generatorTimeframeTable(table, ativo)
    
    index_data = TimeframeTable.index
    
    scaler = createTrainScaler(TimeframeTable)

    TimeframeTable = scaler.transform(TimeframeTable)
    
    nameColumns = []

    for i in range(steps,-1,-1):
        nameColumns.append('Close-{}'.format(i))

    TimeframeTable = pd.DataFrame(TimeframeTable, columns = nameColumns, index = index_data)
    
    
    for day in forecast:
        
        current_info = TimeframeTable.iloc[-1, 1:].to_numpy()
        
        standardCurrentInfo = current_info.reshape(1, steps, 1).astype('float32')
        
        current_forecast = model.predict(standardCurrentInfo, verbose=False).reshape(1,)
        
        new_line = np.concatenate((current_info, current_forecast), axis = 0)
        
        TimeframeTable = pd.concat([TimeframeTable,
                                    pd.DataFrame(new_line.reshape(1, -1),
                                                 columns = nameColumns,
                                                 index = [day])], axis = 0)
        
        
    index_data = TimeframeTable.index  
    
    TimeframeTable = scaler.inverse_transform(TimeframeTable)
    
    TimeframeTable = pd.DataFrame(TimeframeTable, columns = nameColumns, index = index_data)
    
    TimeframeTable.index = pd.to_datetime(TimeframeTable.index)
    
    
    #--------Popula tabela de previsão---------------------------------------------------
    if ativo in ativos:
            for day in forecast:
                tablePrevision.loc[day, ativo] = TimeframeTable.loc[day, 'Close-0']
    #------------------------------------------------------------------------------------
   
    return TimeframeTable

In [29]:
# Popula tabela de log-Retorno, Previsão de Preços e gera gráficos

def fillPrediction(tableLogRet, tablePrevision, nameSetor, setor, model):
    
    lengthTable = len(tableLogRet)
    
    time = datetime.now().strftime('%d-%B-%Ih%Mmin')

    for ativo in setor:

        TimeframeSPAux = fillTableFrame(ativo, tablePrevision, model)

        #-----------Graphic------------------------------------------------------------------------------------------
        if graphic:
            
            outdir = './graphics/{}-{}'.format(nameSetor, time)

            if not os.path.exists(outdir):
                os.mkdir(outdir)
            
            fig, ax = plt.subplots()
            ax.plot(TimeframeSPAux.index[:-len(forecast)], TimeframeSPAux.iloc[:-len(forecast), -1], linewidth=3.0, c = 'b')
            ax.plot(TimeframeSPAux.index[-len(forecast):], TimeframeSPAux.iloc[-len(forecast):, -1], linewidth=3.0, c = 'c', ls = '-')
            ax.legend(['Atual', 'Previsão'])
            ax.set_title('Preço de Fechamento - {}'.format(ativo))
            ax.set(xlabel='Tempo (ano)', ylabel='Preço ($)')
            nameGraphic = '{}.jpg'.format(ativo)
            fullname = os.path.join(outdir, nameGraphic)
            plt.savefig(fullname)
            plt.close(fig)
        #------------------------------------------------------------------------------------------------------------

        
        #--------Popula tabela de Log Retorno------------------------------------------------------------------------
        for n in range(len(forecast)):
            tableLogRet.loc[lengthTable-n-1, ativo] = \
            np.log(TimeframeSPAux.iloc[lengthTable-steps-n-1, -1] / TimeframeSPAux.iloc[lengthTable-steps-n-1-logRetPeriod, -1])
        #------------------------------------------------------------------------------------------------------------

In [30]:
# Cria tabela de Previsão

update = pd.DataFrame(index = pd.to_datetime(forecast), columns = ativos) \
    .reset_index().rename(columns={'index': 'Dia'})

tablePrevision = pd.concat([SP500_close, update], axis = 0, ignore_index = True).set_index('Dia')

tablePrevision.index = pd.to_datetime(tablePrevision.index)

In [31]:
# Cria tabela de Log-Retorno vazia 

index_data = pd.to_datetime(SP500_close['Dia'].append(pd.Series(forecast)))

tableLogRet = pd.DataFrame(index = index_data,
                           columns = ativos).reset_index().rename(columns={'index': 'Dia'})

tableLogRet['Dia'] = tableLogRet['Dia'].apply(lambda date: date.strftime('%d/%m'))

In [32]:
for nameSetor, setor in setores.items():

    if trainModel:
        X_treino, X_teste, y_treino, y_teste = preprocessingdata(steps, SP500_close, setor)
        model = create_model()

        model.fit(x = X_treino,
                  y = y_treino,
                  batch_size = batch_size,
                  epochs = epochs,
                  verbose = verbose,
                  validation_data = (X_teste, y_teste),
                  callbacks = callbacks)
        
        #---------Save Model---------------------------
        model.save('saveModel/{}/'.format(nameSetor),
                   overwrite=True,
                   include_optimizer=True,
                   save_format = 'tf')
        #----------------------------------------------

        scoreTrain = model.evaluate(X_treino, y_treino)
        scoreTest = model.evaluate(X_teste, y_teste)

        print('\n\nErro quadrático médio em dados de treinamento: {:.5f}\n\nErro quadrático médio em dados de teste: {:.5f}\n\n'\
            .format(scoreTrain, scoreTest))

        fillPrediction(tableLogRet, tablePrevision, nameSetor, setor, model)
        
    else:
        model = load_model('saveModel/{}/'.format(nameSetor))
        fillPrediction(tableLogRet, tablePrevision, nameSetor, setor, model)


# Resultados

In [33]:
tableLogRet

,Dia,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
0,23/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,14/11,0.013243,0.073846,0.031313,0.027915,-0.019678,-0.010535,0.008324,0.01969,0.026071,...,0.100267,0.038446,-0.018555,0.103174,0.049843,0.019557,0.002741,0.02083,0.023101,0.012843
1275,15/11,-0.000343,0.037645,0.002585,0.018935,-0.021889,-0.022731,0.003374,0.005494,0.019505,...,0.118154,0.022256,-0.021758,0.100735,0.016725,0.005752,0.016664,0.012753,0.021113,0.008809
1276,16/11,-0.004205,0.043843,0.008272,0.019398,-0.02107,-0.021099,-0.000315,0.003369,0.01436,...,0.096784,0.023035,-0.023116,0.095763,0.016817,0.004438,0.009985,0.012836,0.01477,0.00652
1277,17/11,-0.008224,0.03944,0.002833,0.016595,-0.019896,-0.022266,-0.001534,-0.000779,0.010339,...,0.091102,0.020876,-0.027155,0.09216,0.007839,0.000733,0.009507,0.010528,0.012308,0.004884


### Salva Métricas

In [34]:
# Salva tabela de previsão

outdirP = './previsao/{}'.format(datetime.now().strftime('%d-%B-%Ih%Mmin'))

if not os.path.exists(outdirP):
    os.mkdir(outdirP)

fullnameP = os.path.join(outdirP, 'previsao.csv')
fullnamePExcel = os.path.join(outdirP, 'previsao.xlsx')

tablePrevision.to_csv(fullnameP, index = True, decimal = '.', sep=',')  
tablePrevision.to_excel(fullnamePExcel)

In [35]:
# Salva tabela Entregável do Log-Retorno no padrão

outdirLR = './logRetorno/{}'.format(datetime.now().strftime('%d-%B-%Ih%Mmin'))

if not os.path.exists(outdirLR):
    os.mkdir(outdirLR)

fullnameLR = os.path.join(outdirLR, 'logRetorno.csv')
fullnameLRExcel = os.path.join(outdirLR, 'logRetorno.xlsx')

tableLogRet.iloc[-len(forecast):, :].to_csv(fullnameLR, index = False, decimal = '.', sep=',')   
tableLogRet.iloc[-len(forecast):, :].to_excel(fullnameLRExcel)